# 🍍🍅 Fruit & Vegetable Recognition using MobileNet

This notebook trains a deep learning model to classify 36 different types of fruits and vegetables using MobileNetV2 transfer learning.

## 1. Setup and Install Dependencies

In [ ]:
# Install required packages
!pip install -q tensorflow keras pillow numpy matplotlib scikit-learn

## 2. Import Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 3. Download and Setup Dataset

**Choose ONE of these options:**

**Option 1: Load from Google Drive (Recommended if already uploaded)**
- If you have `archive.zip` in your Google Drive at `DATA/archive.zip`
- This is the fastest option!

**Option 2: Using Kaggle API**
- Upload your kaggle.json file
- Dataset downloads automatically

**Option 3: Manual Upload**
- Upload the dataset zip file directly to Colab

In [ ]:
# ============================================
# OPTION 1: Load from Google Drive (RECOMMENDED)
# ============================================
# Uncomment the lines below if you have archive.zip in Google Drive

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Extract dataset from Google Drive
!unzip -q /content/drive/MyDrive/DATA/archive.zip -d dataset
print("✅ Dataset extracted from Google Drive!")

# ============================================
# OPTION 2: Using Kaggle API
# ============================================
# Uncomment the lines below if using Kaggle API

# from google.colab import files
# files.upload()  # Upload your kaggle.json

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition
# !unzip -q fruit-and-vegetable-image-recognition.zip -d dataset
# print("✅ Dataset downloaded from Kaggle!")

# ============================================
# OPTION 3: Manual Upload
# ============================================
# Uncomment the lines below to upload manually

# from google.colab import files
# print("Upload your dataset zip file:")
# uploaded = files.upload()
# import zipfile
# for filename in uploaded.keys():
#     with zipfile.ZipFile(filename, 'r') as zip_ref:
#         zip_ref.extractall('dataset')
# print("✅ Dataset uploaded and extracted!")

# ============================================
# Set dataset paths
# ============================================
DATASET_DIR = 'dataset'
TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
VAL_DIR = os.path.join(DATASET_DIR, 'validation')

print(f"\n📁 Dataset directory: {DATASET_DIR}")
print(f"📁 Training directory: {TRAIN_DIR}")
print(f"📁 Validation directory: {VAL_DIR}")

# Verify dataset structure
if os.path.exists(TRAIN_DIR) and os.path.exists(VAL_DIR):
    train_classes = len(os.listdir(TRAIN_DIR))
    val_classes = len(os.listdir(VAL_DIR))
    print(f"\n✅ Dataset loaded successfully!")
    print(f"   Training classes: {train_classes}")
    print(f"   Validation classes: {val_classes}")
else:
    print("\n⚠️ Warning: Dataset directories not found!")
    print("   Please check the paths and try again.")

## 4. Configuration and Parameters

In [ ]:
# Model parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

# Class labels
labels = {
    0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'bell pepper', 4: 'cabbage', 
    5: 'capsicum', 6: 'carrot', 7: 'cauliflower', 8: 'chilli pepper', 9: 'corn', 
    10: 'cucumber', 11: 'eggplant', 12: 'garlic', 13: 'ginger', 14: 'grapes', 
    15: 'jalepeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce', 19: 'mango', 
    20: 'onion', 21: 'orange', 22: 'paprika', 23: 'pear', 24: 'peas', 
    25: 'pineapple', 26: 'pomegranate', 27: 'potato', 28: 'raddish', 
    29: 'soy beans', 30: 'spinach', 31: 'sweetcorn', 32: 'sweetpotato', 
    33: 'tomato', 34: 'turnip', 35: 'watermelon'
}

NUM_CLASSES = len(labels)
print(f"Number of classes: {NUM_CLASSES}")

## 5. Data Preparation and Augmentation

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation
val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Classes found: {len(train_generator.class_indices)}")

## 6. Visualize Sample Images

In [ ]:
# Display sample images from training set
sample_images, sample_labels = next(train_generator)

plt.figure(figsize=(15, 10))
for i in range(min(9, len(sample_images))):
    plt.subplot(3, 3, i + 1)
    plt.imshow(sample_images[i])
    class_idx = np.argmax(sample_labels[i])
    plt.title(f"Class: {labels[class_idx]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## 7. Build Model using MobileNetV2

In [ ]:
# Load pre-trained MobileNetV2 without top layers
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

## 8. Setup Callbacks

In [ ]:
# Create callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

callbacks = [early_stopping, reduce_lr, checkpoint]

## 9. Train the Model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# Save the final model
model.save('FV.h5')
print("Model saved as FV.h5")

## 10. Plot Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss plot
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 11. Evaluate Model

In [ ]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"\nValidation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

## 12. Prediction Function

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_image(image_path, model):
    """
    Predict the class of a single image
    
    Args:
        image_path: Path to the image file
        model: Trained Keras model
    
    Returns:
        Predicted class name and confidence
    """
    # Load and preprocess image
    img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    # Get class name
    class_name = labels[predicted_class]
    
    # Display image with prediction
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Predicted: {class_name.capitalize()}\nConfidence: {confidence*100:.2f}%")
    plt.axis('off')
    plt.show()
    
    return class_name, confidence

# Example usage (uncomment and provide image path)
# predicted_class, confidence = predict_image('path/to/your/image.jpg', model)
# print(f"Predicted: {predicted_class} with {confidence*100:.2f}% confidence")

## 13. Test with Sample Images

In [ ]:
# Test with random validation images
val_generator.reset()
test_images, test_labels = next(val_generator)

# Make predictions
predictions = model.predict(test_images[:9])

# Display results
plt.figure(figsize=(15, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(test_images[i])
    
    true_class = np.argmax(test_labels[i])
    pred_class = np.argmax(predictions[i])
    confidence = predictions[i][pred_class]
    
    color = 'green' if true_class == pred_class else 'red'
    plt.title(f"True: {labels[true_class]}\nPred: {labels[pred_class]}\n({confidence*100:.1f}%)", 
              color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

## 14. Download Trained Model

In [ ]:
# Download the model file (for Colab)
from google.colab import files

try:
    files.download('FV.h5')
    print("Model downloaded successfully!")
except:
    print("Not running in Colab or download failed. Model saved locally as FV.h5")

## 15. Interactive Web UI in Colab (Gradio)

In [ ]:
# Install Gradio for interactive web UI
!pip install -q gradio

import gradio as gr
from PIL import Image

def predict_fruit_vegetable(image):
    """
    Predict fruit/vegetable from uploaded image
    
    Args:
        image: PIL Image or numpy array
    
    Returns:
        Dictionary with prediction results
    """
    # Ensure image is PIL Image
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)
    
    # Resize and preprocess
    img = image.resize((IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    # Get class name
    class_name = labels[predicted_class].capitalize()
    
    # Determine category
    fruits = ['Apple', 'Banana', 'Bell Pepper', 'Chilli Pepper', 'Grapes', 'Jalepeno', 
              'Kiwi', 'Lemon', 'Mango', 'Orange', 'Paprika', 'Pear', 'Pineapple', 
              'Pomegranate', 'Watermelon']
    
    category = "🍎 Fruit" if class_name in fruits else "🥬 Vegetable"
    
    # Get top 3 predictions
    top_3_idx = np.argsort(predictions[0])[-3:][::-1]
    top_3_results = {
        labels[idx].capitalize(): float(predictions[0][idx]) 
        for idx in top_3_idx
    }
    
    # Format result
    result = f"""
    ### Prediction: {class_name}
    **Category:** {category}
    **Confidence:** {confidence*100:.2f}%
    
    #### Top 3 Predictions:
    """
    for name, conf in top_3_results.items():
        result += f"\n- **{name}**: {conf*100:.2f}%"
    
    return result

# Create Gradio interface
demo = gr.Interface(
    fn=predict_fruit_vegetable,
    inputs=gr.Image(type="pil", label="Upload Fruit or Vegetable Image"),
    outputs=gr.Markdown(label="Prediction Results"),
    title="🍍🍅 Fruit & Vegetable Classifier",
    description="Upload an image of a fruit or vegetable to classify it! Supports 36 different classes.",
    examples=[],  # You can add example image paths here
    theme="soft",
    allow_flagging="never"
)

# Launch the interface
print("🚀 Launching interactive web UI...")
print("📱 The interface will open below and provide a public URL you can share!")
demo.launch(share=True, debug=False)